# Paper on Synthetic Signatures

In [ ]:
%load_ext autoreload

%autoreload 2

# import packages
import coreweb
from coreweb.methods.offwebutils import get_modelkwargs_ranges, offwebfit, get_eventinfo, update_posfig_offweb, create_movie, ropechecker, signaturecheck, signaturecheckfull
from coreweb.dashcore.utils.plotting import check_animation, check_fittingpoints, get_longmove_array
from coreweb.dashcore.utils.utils import load_fit, cart2sphere, sphere2cart, round_to_hour_or_half, get_iparams_live, process_coordinates
from coreweb.dashcore.assets.config_sliders import modelslidervars, magslidervars
from coreweb.dashcore.app import update_launch_label, generate_graphstore
from coreweb.dashcore.pages.Start import update_alert_for_init

import coreweb.dashcore.utils.heliocats as hc

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly_express as px
import plotly.figure_factory as ff
from plotly.validators.scatter.marker import SymbolValidator
import plotly.io as pio

import pandas as pd
from IPython.display import display, HTML

import os

import datetime
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import and process data

reference_frame = "RTN" # switch to HEEQ if needed 
idd = 'ICME_SYN_CUSTOM'


eventinfo = get_eventinfo(idd, purelysyn=True)
graphstore, posstore, _ = generate_graphstore(eventinfo, reference_frame, {})

In [ ]:
# choose from the following plot options: "Title", "Latitudinal Grid", "Longitudinal Grid","Trajectories","Synthetic Event","Catalog Event","AU axis","Trajectories","Parker Spiral","Catalog Event", "Timer"

plot_options = [
    "Longitudinal Grid",
    "AU axis",
    "Synthetic Event",
    "dotted", # if you want the grid to be dotted, use dashed or remove if not
    ]

# choose from the following spacecraft: "SOLO", "PSP", "BEPI", "Wind","STEREO-A", "SYN"
spacecraftoptions = None # ["SYN"]

# choose from the following bodies: "Sun", "Mercury", "Venus","Earth"
bodyoptions = ["Sun"]

firstfield = 50.
secondfield = 5.
nameadd = ''

view_legend_insitu = True
insitu = False
positions = True
plottheme = 'light-simple' # 'dark', 'light', 'dark-simple', 'light-simple'
currenttimeslider = 55
launchlabel = None
rinput = 0.8
lonput = 0
latput = 0

deltatime = 100
longmove =  0 #0.03

camera = [1.0,-30,0] #'auto' give r, lon and lat
# this function creates the figure

longmove_array = get_longmove_array(longmove, rinput,lonput,latput, graphstore)

### Show insitu data and 3D Plot 16 SPACESCRAFT SYN

In [ ]:
#top down view field line
camera = [0,0,1.5]
plot_options = ["Synthetic Event","dotted"]


modelstatevars1 = [0., # Longitude
                  0., # Latitude
                  0., # Inclination
                  0.3, # Diameter 1 AU 
                  2., # Aspect Ratio
                  20., # Launch Radius
                  600., # Launch Velocity
                  1.14, # Expansion Rate
                  1.00, # Background Drag
                  500., # Background Velocity
                  firstfield, # T_Factor
                  1.64, # Magnetic Decay Rate
                  25., # Magnetic Field Strength 1 AU
                  ]

tracingfield1 = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1, addfield = True, secondfield=secondfield)

# top down view field line
tracingfield1.write_image("topdownfield.pdf", width = 1000 * 3, height = 1000 * 3, scale=3)

checkanim = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)
#checkanim.write_html("checkanim.html")

syncolors = [
'#009e74',
'#56b3e9',
'#e66000',
'#cc79a7',

# '#C02469',
# '#F14091',
# '#4CC12E',
# '#33A6EC',
# '#1D7DC0',

#    '#44AA99', # greenish
#    '#CC6677', # pinkish
#    '#882255', # darkred
#    '#332288', # darkblue
]
synshapes = [
    'cross', 
    'diamond',
    'square', 
    'circle-open',
    'circle' #  ['circle', 'circle-open', 'cross', 'diamond','diamond-open', 'square', 'square-open', 'x']
]

lons = [-30, 0 , 30, 45, 60]

lats = [15, 5, 0, -15]

namecount = 0

for j, lon in enumerate(lons):
    for k,lat in enumerate(lats):
        x,y,z = sphere2cart(float(rinput), np.deg2rad(-float(lat)+90), np.deg2rad(float(lon)))

        checkanim.add_trace(
            go.Scatter3d(
                x=[x], y=[y], z=[z],
                mode='markers', 
                marker=dict(size=6, 
                            symbol=synshapes[j],
                            color=syncolors[k]),
                name="lon: "+str(lon)+', lat: ' + str(lat),
                legendgroup = '1',
                showlegend=True,
            ), row=1, col=1)
        
        namecount += 1

tracingfield1.show(config = {
  'toImageButtonOptions': {
    'format': 'svg'
  }
})

checkanim.show(config = {
  'toImageButtonOptions': {
    'format': 'svg'
  }
})


modelstatevars1 = [0., # Longitude
                  0., # Latitude
                  90., # Inclination
                  0.3, # Diameter 1 AU 
                  2., # Aspect Ratio
                  20., # Launch Radius
                  600., # Launch Velocity
                  1.14, # Expansion Rate
                  1.00, # Background Drag
                  500., # Background Velocity
                  secondfield, # T_Factor
                  1.64, # Magnetic Decay Rate
                  25., # Magnetic Field Strength 1 AU
                  ]

iparams = get_iparams_live(*modelstatevars1)
iparamslist, inccheck = ropechecker(iparams)


checkanim = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)

tracingfield2 = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1, addfield = True, secondfield = secondfield)

lons = [-15, 0, 5, 15]

lats = [45, 30 , 0, -30]

namecount = 0

for j, lon in enumerate(lons):
    for k,lat in enumerate(lats):
        x,y,z = sphere2cart(float(rinput), np.deg2rad(-float(lat)+90), np.deg2rad(float(lon)))

        checkanim.add_trace(
            go.Scatter3d(
                x=[x], y=[y], z=[z],
                mode='markers', 
                marker=dict(size=6, 
                            symbol=synshapes[j],
                            color=syncolors[k]),
                name="lon: "+str(lon)+', lat: ' + str(lat),
                legendgroup = '1',
                showlegend=True,
            ), row=1, col=1)
        
        namecount += 1

tracingfield2.show(config = {
  'toImageButtonOptions': {
    'format': 'svg'
  }
})

checkanim.show(config = {
  'toImageButtonOptions': {
    'format': 'svg'
  }
})

In [ ]:
#########   LOW INC LOW TWIST


plot_options = [
    "Longitudinal Grid",
    "AU axis",
    "Synthetic Event",
    "dotted", # if you want the grid to be dotted, use dashed or remove if not
    ]
    
    

##### model parameters

modelstatevars1 = [0., # Longitude
                  0., # Latitude
                  0., # Inclination
                  0.3, # Diameter 1 AU 
                  2., # Aspect Ratio
                  20., # Launch Radius
                  600., # Launch Velocity
                  1.14, # Expansion Rate
                  1.00, # Background Drag
                  500., # Background Velocity
                  secondfield, # T_Factor
                  1.64, # Magnetic Decay Rate
                  25., # Magnetic Field Strength 1 AU
                  ]

iparams = get_iparams_live(*modelstatevars1)
iparamslist, inccheck = ropechecker(iparams)

fluxtypeslow = ['SWN_low','NES_low', 'NWS_low','SEN_low']

fluxtypeshigh = ['WNE_high','ESW_high', 'ENW_high','WSE_high']

checkanim = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)
#checkanim.write_html("checkanim.html")



lons = [-30, 0 , 30, 45, 60]

lats = [15, 5, 0, -15]

namecount = 0

for j, lon in enumerate(lons):
    for k,lat in enumerate(lats):
        x,y,z = sphere2cart(float(rinput), np.deg2rad(-float(lat)+90), np.deg2rad(float(lon)))

        checkanim.add_trace(
            go.Scatter3d(
                x=[x], y=[y], z=[z],
                mode='markers', 
                marker=dict(size=4, 
                            symbol=synshapes[j],
                            color=syncolors[k]),
                name="SYN_"+str(namecount),
                legendgroup = '1',
                showlegend=True,
            ), row=1, col=1)
        
        namecount += 1

# checkanim.show(config = {
#   'toImageButtonOptions': {
#     'format': 'svg'
#   }
# })
        


for i, namedir in enumerate(fluxtypeslow):

    
    iparams = iparamslist[i]

    print(iparams)
    print(namedir)

    t_launch = datetime.datetime(2012,12,21,6)
    model_obj = coreweb.ToroidalModel(t_launch, **iparams) # model gets initialized
    model_obj.generator()  
    checkanimany = check_animation(None, 
                                None,
                                plottheme, 
                                graphstore, 
                                reference_frame, 
                                rinput, lonput, latput, 
                                currenttimeslider, 
                                eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)

    signaturecheckfull(lats, lons, model_obj, rinput, 'low_inc_low_twist_' + nameadd + namedir, checkanimany, graphstore, t_launch)



############# HIGH INC LOW TWIST

##### model parameters

modelstatevars1 = [0., # Longitude
                  0., # Latitude
                  90., # Inclination
                  0.3, # Diameter 1 AU 
                  2., # Aspect Ratio
                  20., # Launch Radius
                  600., # Launch Velocity
                  1.14, # Expansion Rate
                  1.00, # Background Drag
                  500., # Background Velocity
                  secondfield, # T_Factor
                  1.64, # Magnetic Decay Rate
                  25., # Magnetic Field Strength 1 AU
                  ]

iparams = get_iparams_live(*modelstatevars1)
iparamslist, inccheck = ropechecker(iparams)


checkanim = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)



lons = [-15, 0, 5, 15]

lats = [45, 30 , 0, -30]

namecount = 0

for j, lon in enumerate(lons):
    for k,lat in enumerate(lats):
        x,y,z = sphere2cart(float(rinput), np.deg2rad(-float(lat)+90), np.deg2rad(float(lon)))

        checkanim.add_trace(
            go.Scatter3d(
                x=[x], y=[y], z=[z],
                mode='markers', 
                marker=dict(size=4, 
                            symbol=synshapes[j],
                            color=syncolors[k]),
                name="SYN_"+str(namecount),
                legendgroup = '1',
                showlegend=True,
            ), row=1, col=1)
        
        namecount += 1

# checkanim.show(config = {
#   'toImageButtonOptions': {
#     'format': 'svg'
#   }
# })


for i, namedir in enumerate(fluxtypeshigh):

    
    iparams = iparamslist[i]

    print(iparams)
    print(namedir)

    t_launch = datetime.datetime(2012,12,21,6)
    model_obj = coreweb.ToroidalModel(t_launch, **iparams) # model gets initialized
    model_obj.generator()  
    checkanimany = check_animation(None, 
                                None,
                                plottheme, 
                                graphstore, 
                                reference_frame, 
                                rinput, lonput, latput, 
                                currenttimeslider, 
                                eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)

    signaturecheckfull(lats, lons, model_obj, rinput, 'high_inc_low_twist_' + nameadd + namedir, checkanimany, graphstore, t_launch)




#########   LOW INC HIGH TWIST



##### model parameters

modelstatevars1 = [0., # Longitude
                  0., # Latitude
                  0., # Inclination
                  0.3, # Diameter 1 AU 
                  2., # Aspect Ratio
                  20., # Launch Radius
                  600., # Launch Velocity
                  1.14, # Expansion Rate
                  1.00, # Background Drag
                  500., # Background Velocity
                  firstfield, # T_Factor
                  1.64, # Magnetic Decay Rate
                  25., # Magnetic Field Strength 1 AU
                  ]

iparams = get_iparams_live(*modelstatevars1)
iparamslist, inccheck = ropechecker(iparams)

fluxtypeslow = ['SWN_low','NES_low', 'NWS_low','SEN_low']

fluxtypeshigh = ['WNE_high','ESW_high', 'ENW_high','WSE_high']

checkanim = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)
#checkanim.write_html("checkanim.html")


lons = [-30, 0 , 30, 45, 60]

lats = [15, 5, 0, -15]

namecount = 0

for j, lon in enumerate(lons):
    for k,lat in enumerate(lats):
        x,y,z = sphere2cart(float(rinput), np.deg2rad(-float(lat)+90), np.deg2rad(float(lon)))

        checkanim.add_trace(
            go.Scatter3d(
                x=[x], y=[y], z=[z],
                mode='markers', 
                marker=dict(size=4, 
                            symbol=synshapes[j],
                            color=syncolors[k]),
                name="SYN_"+str(namecount),
                legendgroup = '1',
                showlegend=True,
            ), row=1, col=1)
        
        namecount += 1

# checkanim.show(config = {
#   'toImageButtonOptions': {
#     'format': 'svg'
#   }
# })
        


for i, namedir in enumerate(fluxtypeslow):

    
    iparams = iparamslist[i]

    print(iparams)
    print(namedir)

    t_launch = datetime.datetime(2012,12,21,6)
    model_obj = coreweb.ToroidalModel(t_launch, **iparams) # model gets initialized
    model_obj.generator()  
    checkanimany = check_animation(None, 
                                None,
                                plottheme, 
                                graphstore, 
                                reference_frame, 
                                rinput, lonput, latput, 
                                currenttimeslider, 
                                eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)

    signaturecheckfull(lats, lons, model_obj, rinput, 'low_inc_high_twist_' + nameadd + namedir, checkanimany, graphstore, t_launch)



############# HIGH INC HIGH TWIST

##### model parameters

modelstatevars1 = [0., # Longitude
                  0., # Latitude
                  90., # Inclination
                  0.3, # Diameter 1 AU 
                  2., # Aspect Ratio
                  20., # Launch Radius
                  600., # Launch Velocity
                  1.14, # Expansion Rate
                  1.00, # Background Drag
                  500., # Background Velocity
                  firstfield, # T_Factor
                  1.64, # Magnetic Decay Rate
                  25., # Magnetic Field Strength 1 AU
                  ]

iparams = get_iparams_live(*modelstatevars1)
iparamslist, inccheck = ropechecker(iparams)


checkanim = check_animation(None, 
                            None,
                            plottheme, 
                            graphstore, 
                            reference_frame, 
                            rinput, lonput, latput, 
                            currenttimeslider, 
                            eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)



lons = [-5, 0, 5, 15]

lats = [45, 30 , 0, -30]

namecount = 0

for j, lon in enumerate(lons):
    for k,lat in enumerate(lats):
        x,y,z = sphere2cart(float(rinput), np.deg2rad(-float(lat)+90), np.deg2rad(float(lon)))

        checkanim.add_trace(
            go.Scatter3d(
                x=[x], y=[y], z=[z],
                mode='markers', 
                marker=dict(size=4, 
                            symbol=synshapes[j],
                            color=syncolors[k]),
                name="SYN_"+str(namecount),
                legendgroup = '1',
                showlegend=True,
            ), row=1, col=1)
        
        namecount += 1

# checkanim.show(config = {
#   'toImageButtonOptions': {
#     'format': 'svg'
#   }
# })


for i, namedir in enumerate(fluxtypeshigh):

    
    iparams = iparamslist[i]

    print(iparams)
    print(namedir)

    t_launch = datetime.datetime(2012,12,21,6)
    model_obj = coreweb.ToroidalModel(t_launch, **iparams) # model gets initialized
    model_obj.generator()  
    checkanimany = check_animation(None, 
                                None,
                                plottheme, 
                                graphstore, 
                                reference_frame, 
                                rinput, lonput, latput, 
                                currenttimeslider, 
                                eventinfo, launchlabel, plot_options, spacecraftoptions, bodyoptions,  insitu, positions, view_legend_insitu, camera.copy(), posstore, *modelstatevars1)

    signaturecheckfull(lats, lons, model_obj, rinput, 'high_inc_high_twist_' + nameadd  + namedir, checkanimany, graphstore, t_launch)